In [1]:
import os
import django
from django.conf import settings


In [2]:
os.environ['DJANGO_SETTINGS_MODULE'] = 'sample_project.settings'

In [3]:
django.setup()

In [4]:
from sample_app.models import Country, Author, Book

### Creating Objects

**An OOP Interface:**

In [5]:
a = Author()
a.name = 'George Orwell'
a.country_id = 2

In [6]:
a.name

'George Orwell'

In [7]:
a.country_id

2

In [8]:
a.save()

**An even more familiar OOP interface**

In [9]:
a = Author(name='Ernest Hemingway', country_id=1)

In [10]:
a.save()

### Reading Objects

Operations at "table-level" are performed with a special attribute named `objects`:

**`all()`: select star**

In [11]:
Author.objects.all()

<QuerySet [<Author: Edgar Allan Poe>, <Author: Mark Twain>, <Author: Jane Austen>, <Author: Arthur Conan Doyle>, <Author: Jorge Luis Borges>, <Author: George Orwell>, <Author: Ernest Hemingway>]>

In [12]:
author = Author.objects.all()[0]

In [13]:
author.name

'Edgar Allan Poe'

In [14]:
for author in Author.objects.all():
    print("Author: {}, from {}".format(author.name, author.country_id))

Author: Edgar Allan Poe, from 1
Author: Mark Twain, from 1
Author: Jane Austen, from 2
Author: Arthur Conan Doyle, from 4
Author: Jorge Luis Borges, from 3
Author: George Orwell, from 2
Author: Ernest Hemingway, from 1


**`filter`: A supercharged WHERE**

In [15]:
authors = Author.objects.filter(country_id=1)

In [16]:
for author in authors:
    print(author.name)

Edgar Allan Poe
Mark Twain
Ernest Hemingway


**Getting a single object with `filter`**

In [17]:
authors = Author.objects.filter(name='Mark Twain')
authors

<QuerySet [<Author: Mark Twain>]>

In [18]:
for author in authors:
    print(author.name)

Mark Twain


In [19]:
author = authors[0]

In [20]:
author.name

'Mark Twain'

In [21]:
author.country_id

1

**`get`: the correct solution**

In [22]:
author = Author.objects.get(name='Mark Twain')
author

<Author: Mark Twain>

In [23]:
author.name

'Mark Twain'

**`get`: fails if it can't find an object**

In [24]:
Author.objects.get(name='Not a real author')

DoesNotExist: Author matching query does not exist.

And the exception is `DoesNotExist`. To prevent it, you must surround it with a `try/except` block:

In [25]:
try:
    Author.objects.get(name='Not a real author')
except Author.DoesNotExist:
    print("Author doesn't exist")

Author doesn't exist


Or we can also use **`exists()`** (combination with `filter()`) to prevent the lookup:

In [26]:
Author.objects.filter(name='Not a real author').exists()

False

**`get`: also fails if MULTIPLE objects satisfy the lookup`**

In [27]:
Author.objects.get(country_id=1)

MultipleObjectsReturned: get() returned more than one Author -- it returned 3!

**`count()` to prevent these errors**

In [28]:
Author.objects.filter(country_id=1).count()

3

**Combining books, authors and countries (old JOIN)**

In [29]:
book = Book.objects.get(id=1)

In [30]:
book.get_author()

<Author: Jane Austen>

In [31]:
author = book.get_author()

In [32]:
author.name

'Jane Austen'

In [33]:
author.get_country()

<Country: England>

### More advanced filtering

`filter` allows you to define comparison operators like the ones in SQL (`LIKE`, `>`, `<`, etc), but it has a special syntax.

**`=` equals**

In [6]:
Book.objects.filter(title='A Study in Scarlet')

<QuerySet [<Book: A Study in Scarlet(4.60)>]>

In [7]:
Book.objects.filter(title='does not exist')

<QuerySet []>

**`<`, `>`, `<=`, `>=`: comparison operators**

`<` _lower than_:

In [11]:
Book.objects.filter(popularity__lt=4.6)

<QuerySet [<Book: Emma(3.50)>]>

`<=` _lower than or equals to_:

In [10]:
Book.objects.filter(popularity__lte=4.6)

<QuerySet [<Book: A Study in Scarlet(4.60)>, <Book: Emma(3.50)>]>

`>` _greater than_:

In [13]:
Book.objects.filter(popularity__gt=4.6)

<QuerySet [<Book: Pride & Prejudice(5.00)>]>

`>=` _greater than or equals to_:

In [15]:
Book.objects.filter(popularity__gte=4.6)

<QuerySet [<Book: Pride & Prejudice(5.00)>, <Book: A Study in Scarlet(4.60)>]>

**Membership, the `in` operator**

In [24]:
Author.objects.filter(country_id__in=[1, 2])

<QuerySet [<Author: Edgar Allan Poe>, <Author: Mark Twain>, <Author: Jane Austen>, <Author: George Orwell>, <Author: Ernest Hemingway>]>

**Range search**

In [31]:
Book.objects.filter(popularity__range=[4, 5])

<QuerySet [<Book: Pride & Prejudice(5.00)>, <Book: A Study in Scarlet(4.60)>]>

**String filters**

`iexact` _ignore case_:

In [17]:
Book.objects.filter(title__iexact='a study in scarlet')

<QuerySet [<Book: A Study in Scarlet(4.60)>]>

`startswith` _starts with_:

In [22]:
Book.objects.filter(title__startswith='A Study')

<QuerySet [<Book: A Study in Scarlet(4.60)>]>

`istartswith` _starts with, ignore case_:

In [23]:
Book.objects.filter(title__istartswith='a study')

<QuerySet [<Book: A Study in Scarlet(4.60)>]>

`contains` _contains the following chars_:

In [27]:
Book.objects.filter(title__contains='Study')

<QuerySet [<Book: A Study in Scarlet(4.60)>]>

`icontains` _contains the following chars, ignore case_:

In [28]:
Book.objects.filter(title__icontains='study')

<QuerySet [<Book: A Study in Scarlet(4.60)>]>

See more here: https://docs.djangoproject.com/en/2.0/topics/db/queries/#field-lookups (and [reference](https://docs.djangoproject.com/en/2.0/ref/models/querysets/#field-lookups))

### Ordering results, `order_by`

**Ascending, default**

In [32]:
Book.objects.all().order_by('popularity')

<QuerySet [<Book: Emma(3.50)>, <Book: A Study in Scarlet(4.60)>, <Book: Pride & Prejudice(5.00)>]>

**Descending**

In [33]:
Book.objects.all().order_by('-popularity')

<QuerySet [<Book: Pride & Prejudice(5.00)>, <Book: A Study in Scarlet(4.60)>, <Book: Emma(3.50)>]>